# Task 4

## 1 
Factorize n = 275621053. You can assume that n = pq, where p−q is relatively small. Show
your calculation steps.


Since n-q is relatively small, these numbers must be close to each other. That means that they must also be close to the square root of n. I will start at sqrt(n) and go upwards/downwards on p/q (based on the product compared to n), and find two numbers that multiply to n

In [15]:
import math

n = 275621053
p, q = int(math.sqrt(n)), int(math.sqrt(n))
while p * q != n:
  if p * q < n:
    p += 1
  else:
    q -= 1
print("n =", n)
print("p =", p, "q =", q)
print("p * q =", p * q)

n = 275621053
p = 17021 q = 16193
p * q = 275621053


## Task 2

### a
Alice wants to set up her RSA encryption with private key (n, d) with n = pq, using two
primes p and q, and private key d = 3. She chooses p = 1283, but wonders which of the
following choices for q she should use (NB! They are all prime numbers):
$$ 1307, 1879, 2003, 2027 $$

Explain why she should use q = 2027 for the system to work and to be most secure. For
the weak choices of q, name an effective attack to factorize n (of course, these numbers
are far too small to be secure, so consider the security in relative terms.) <hr>


Since p = 1283, we want a number that is not so close to this. This means that 1307 is not an option. We then look at the other options for q. Both  (p-1) and (q-1) should not have small factors (other than 2 obviously). This is so that we make it more secure against Pollard's p-1 algorithm. We look at the factors for q-1:

In [16]:
def print_factors(x):
   print("The factors of",x,"are:")
   for i in range(2, int(x/2 + 1)):
       if x % i == 0:
           print(i)

numbers = [1879, 2003, 2027]
for number in numbers:
  print_factors(number-1)

The factors of 1878 are:
2
3
6
313
626
939
The factors of 2002 are:
2
7
11
13
14
22
26
77
91
143
154
182
286
1001
The factors of 2026 are:
2
1013


Here we see that 2026 (2027 - 1) is 2*1013, which is a prime multiplied with 2. This makes 2027 a good choice for q, since it's also not so similar to q (1283)

### b
Find the corresponding public key e using the extended Euclidean algorithm. Write a
program to do the calculation.<hr>
Calculate the multiplicative inverse d to e
modulo (𝑝 − 1) (𝑞 − 1) , 𝑠𝑜
𝑑𝑒 ≡ 1 𝑚𝑜𝑑 (𝑝 − 1)(𝑞 − 1)


In [3]:
def extended_euclidean(a, b):
    if a == 0:
        return b, 0, 1
    else:
        gcd, x1, y1 = extended_euclidean(b % a, a)
        # Update x and y using results of recursive call
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y
     

In [25]:
p = 1283
q = 2027
d = 3
phi_n = (p-1)*(q-1)
gcd, e, y = extended_euclidean(d, (p-1)*(q-1))
if e < 0:
    e += phi_n # We want a positive value for e
print("Solution to gcd(a, b) = ax + by, with a=e, x=d=3 and b=(p-1)(q-1)")
print("gcd=", gcd, "e=", e, "y=", y)
print("e=", e, "is the modular inverse of d mod phi(n), and our public key component")
print("d*e - y*phi(n) = ", d*e - 2*(p-1)*(q-1))

Solution to gcd(a, b) = ax + by, with a=e, x=d=3 and b=(p-1)(q-1)
gcd= 1 e= 1731555 y= 1
e= 1731555 is the modular inverse of d mod phi(n), and our public key component
d*e - y*phi(n) =  1


### c
Encrypt the message 111 using repeated squaring. Implement the algorithm yourself.
<hr>

N ≡ M<sup>e</sup> mod(n) <br>
https://www.cs.drexel.edu/~popyack/IntroCS/HW/RSAWorksheet.html

In [27]:
import math

def find_exponent(e, pows):
    bin_exp = bin(e)[2:]  # Convert the exponent to binary string
    for i, bit in enumerate(reversed(bin_exp)):
        if bit == '1':
            pows.append(i)

def encrypt_rsa(m, e, n):
    pows = []
    find_exponent(e, pows)
    calc = m % n
    res = 1  # Initialize to 1 for multiplication
    for i in range(max(pows) + 1):
        if i in pows:
            res = (res * calc) % n
        calc = calc**2 % n
    return res

def decrypt_rsa(c, d, n):
    return pow(c, d, n)


In [28]:
p = 1283
q = 2027
n = p*q
d = 3
e = 1731555
m = 111

print("n=", n, "e=", e, "m=", m)
c = encrypt_rsa(m, e, n)
print("111 encrypted becomes: ", c)
print(c, "decrypted becomes", decrypt_rsa(c, d, n))

n= 2600641 e= 1731555 m= 111
111 encrypted becomes:  1509208
1509208 decrypted becomes 111


## 3
### a
 Let n = 1829 and B = 5. Find a prime factor of n by using Pollard (p − 1) attack.

In [29]:
def pollard(n, B):
    a = 2
    for j in range(2, B + 1):
        a = pow(a, j, n)  
        d = math.gcd(a - 1, n)  
        if 1 < d < n: 
            return d
    return None  

In [39]:
n = 1829
B = 5
p = int(pollard(n, B))
print("n=", n, "B=", B)
print("factor of n is: ", p)
print("n = " + str(n) + " = " + str(p) + " * " + str(int(n/p)))


n= 1829 B= 5
factor of n is:  31
n = 1829 = 31 * 59


### b 
Let n = 18779. Using Pollard (p−1), how small B can be used for the attack to be successful (Use knowledge of the factorizations of n.) You do not need to find the facotirzation.
<hr>

In [38]:
n = 18779
for i in range(10):
  B = i+1
  p = (pollard(n, B))
  if(p == None):
    continue
  print("n=", n, "B=", B)
  print("factor of n is: ", p)
  print("n = " + str(n) + " = " + str(p) + " * " + str(int(n/p)))
  break

print("We see that with B = 7, we find a factor of n")

n= 18779 B= 7
factor of n is:  211
n = 18779 = 211 * 89
We see that with B = 7, we find a factor of n


## 4
Show that encryption in RSA has the following property:
$$e_K(x1)e_K(x2) mod n = e_K(x1x2) mod n $$ 
<hr/>
using p = 1287, q = 2027, d = 3 and e = 868479, x1 = 111, x2 = 234

In [36]:
p = 1287
q = 2027
n = p*q
d = 3
e = 1731555
x1 = 111
x2 = 234

e_x1 = encrypt_rsa(x1, e, n)
e_x2 = encrypt_rsa(x2, e, n)

e_x = (e_x1 * e_x2) % n

e_x1x2 = encrypt_rsa(x1*x2, e, n)

print("n=", n, "e=", e, "x1=", x1, "x2=", x2)

print("ek(x1)*ek(x2) % n = ", e_x)
print("ek(x1*x2) % n = ", e_x1x2)

n= 2608749 e= 1731555 x1= 111 x2= 234
ek(x1)*ek(x2) % n =  245466
ek(x1*x2) % n =  245466


### b
Show how RSA is vulnerable to chosen cipher text attack: 

For ciphertext y, then
Eva can choose some r ̸≡ 1 mod n, and construct y
′ = y · r
e
. 

If she then knows the
decryption x′ = dK(y'), show how she can calculate x = dK(y)

(Hint: She can also
calculate r
−1 mod n)

In [33]:
x = 321
e = 1731555
p = 1283
q = 2027
n = p*q
y = encrypt_rsa(x, e, n)

print("n=", n, "e=", e)
print("y=", y)
print("Goal: find plaintext x for ciphertext y\n")
r = 5
print("selecting r=", r, "such that gcd(r, n) = 1 and r mod n != 1")
print("r mod n = ", r % n, "!= 1")
print("gcd(r, n) = ", math.gcd(r, n), "== 1, meaning they are coprime\n")

y_ = (y * pow(r, e, n)) % n
print("y' = y * r^e mod n = ", y_)
x_ = decrypt_rsa(y_, d, n)
print("x' = y' ^ d mod n = ", x_)
print("This means that x' = x * r mod n")
print("We can find x by finding the modular inverse of r mod n")
print("r=", r, "n=", n)
gcd, r_inv, y = extended_euclidean(r, n)
if r_inv < 0:
    r_inv += n 
print("Modular inverse of r mod n is: ", r_inv)
x_pred = (x_ * r_inv) % n
print("x = x' * r_inv mod n = ", x_pred)
print("\nWe have now constructed x from y, without knowing the private key d, but getting y' decrypted")

n= 2600641 e= 1731555
y= 1656239
Goal: find plaintext x for ciphertext y

selecting r= 5 such that gcd(r, n) = 1 and r mod n != 1
r mod n =  5 != 1
gcd(r, n) =  1 == 1, meaning they are coprime

y' = y * r^e mod n =  2432486
x' = y' ^ d mod n =  1605
This means that x' = x * r mod n
We can find x by finding the modular inverse of r mod n
r= 5 n= 2600641
Modular inverse of r mod n is:  2080513
x = x' * r_inv mod n =  321

We have now constructed x from y, without knowing the private key d, but getting y' decrypted


## 5
Alice and Bob want to have an common key using Diffie-Hellmann key exchange. They
agree on using the prime 101, and base n = 3. Alice choosed her secret a = 33, and Bob chooses
b = 65. <hr>
### a
Write a program that prints out all the powers 3<sup>i</sup>
for i = 1, ..., 100. Do the same for 5<sup>i</sup>
.
What is a major difference between these two sequences? <br>
We can see that all powers of 5 will end in 5, whereas this varies with the powers of 3




In [34]:
print("i", "3^i", "5^i")
for i in range(1, 100+1):
  if(i<10):
    print(i, end="  ")
  else:
    print(i, end=" ")
  print(float(math.pow(3,i)), end=" ")
  print(float(math.pow(5,i)))


i 3^i 5^i
1  3.0 5.0
2  9.0 25.0
3  27.0 125.0
4  81.0 625.0
5  243.0 3125.0
6  729.0 15625.0
7  2187.0 78125.0
8  6561.0 390625.0
9  19683.0 1953125.0
10 59049.0 9765625.0
11 177147.0 48828125.0
12 531441.0 244140625.0
13 1594323.0 1220703125.0
14 4782969.0 6103515625.0
15 14348907.0 30517578125.0
16 43046721.0 152587890625.0
17 129140163.0 762939453125.0
18 387420489.0 3814697265625.0
19 1162261467.0 19073486328125.0
20 3486784401.0 95367431640625.0
21 10460353203.0 476837158203125.0
22 31381059609.0 2384185791015625.0
23 94143178827.0 1.1920928955078124e+16
24 282429536481.0 5.960464477539062e+16
25 847288609443.0 2.9802322387695315e+17
26 2541865828329.0 1.4901161193847657e+18
27 7625597484987.0 7.450580596923828e+18
28 22876792454961.0 3.725290298461914e+19
29 68630377364883.0 1.862645149230957e+20
30 205891132094649.0 9.313225746154785e+20
31 617673396283947.0 4.6566128730773924e+21
32 1853020188851841.0 2.3283064365386964e+22
33 5559060566555523.0 1.164153218269348e+23
34 1.6677

In [35]:
d_prime = 101
base_n = 3
alice_a = 33
bob_b = 65

a1 = pow(base_n, alice_a, d_prime)
b1 = pow(base_n, bob_b, d_prime)
k1 = pow(b1, alice_a, d_prime)
k2 = pow(a1, bob_b, d_prime)
print("p=", d_prime, "n=", base_n, "a=", alice_a, "b=", bob_b)
print("a1=", a1, "b1=", b1)
print("k= b1^a mod p = ", k1)
print("k= a1^b mod p = ", k2)
print("Common key is", k1)

p= 101 n= 3 a= 33 b= 65
a1= 61 b1= 62
k= b1^a mod p =  32
k= a1^b mod p =  32
Common key is 32
